In [109]:
import numpy as np 
from qiskit import QuantumRegister, ClassicalRegister,QuantumCircuit, transpile
from qiskit.providers.aer import QasmSimulator

In [181]:
def num_to_binary(n,N):
    # n is the number to be represented in binary
    # N is the number of digits of the rapresentation
    #es: num_to_binary(5,4) = 0101
    Nbits=2**N
    if n>=Nbits: return 0
    return bin(n+2*Nbits)[4:]


def HammingCircuit(N):
    #Returns a circuit with just the qbits labeled as the parity and signal
    registers=[]
    for i in range(2**N):
        prefix='s' #s stands for signal
        if i==0 or np.log2(i)==int(np.log2(i)): prefix='p' #c stands for parity
        registers.append(QuantumRegister(1,prefix+num_to_binary(i, N)))
    #registers.append(ClassicalRegister(2**N+N))
    circuit=QuantumCircuit(*registers,QuantumRegister(N,'anc')) #circuit already with ancillas
    #circit=QuantumCircuit(*registers)
    circuit.N=N
    return circuit


def xor(N):
    circ=HammingCircuit(N)
    nqubits=2**N
    for i in range(1,nqubits):
        for j in range(0,N):
            if i & 2**j == 2**j: circ.cx(i,nqubits+j)
    return circ.to_gate(label='Initialize')

def correct(N):
    circ=HammingCircuit(N)
    nqubits=2**N
    count=np.zeros(N)
    for i in range(1,nqubits):
        count=count+1
        for j in range(N):
            count[j]=count[j]%(2**j)
            if count[j]==0 and i!=1: circ.x(nqubits+j)
            if i==1 and j!=0: circ.x(nqubits+j)
        circ.mct([*range(nqubits,nqubits+N)],i)
    return circ.to_gate(label='Correction')

def HammingGate0(N):
    circ=HammingCircuit(N)
    circ.append(xor(N),[*range(2**N+N)])
    circ.append(correct(N),[*range(2**N+N)])
    return circ.to_gate(label='Hamming0')

Initializing Ancillas

In [182]:
N=3
circ=QuantumCircuit(2**N+2*N)
asd=HammingGate0(N)
print(type(asd))
circ.append(asd,[*range(2**N+N)])
circ.draw()

<class 'qiskit.circuit.gate.Gate'>


┌────────────┐
 q_0: ┤0           ├
      │            │
 q_1: ┤1           ├
      │            │
 q_2: ┤2           ├
      │            │
 q_3: ┤3           ├
      │            │
 q_4: ┤4           ├
      │            │
 q_5: ┤5  Hamming0 ├
      │            │
 q_6: ┤6           ├
      │            │
 q_7: ┤7           ├
      │            │
 q_8: ┤8           ├
      │            │
 q_9: ┤9           ├
      │            │
q_10: ┤10          ├
      └────────────┘
q_11: ──────────────
                    
q_12: ──────────────
                    
q_13: ──────────────

In [148]:
print(3)

3
